In [ ]:
import pandas as pd
import numpy as np
import datetime
import math
import os

import numpy as np
import pandas as pd

from wfdb.io.annotation import Annotation, wrann
from wfdb.io.record import Record, wrsamp




def pandas_to_wfdb(
    file_name,
    fs,
    units,
    fmt=None,
    adc_gain=None,
    baseline=None,
    samps_per_frame=None,
    counter_freq=None,
    base_counter=None,
    base_time=None,
    base_date=None,
    comments=None,
    sig_name=None,
    dat_file_name=None,
    skew=None,
    byte_offset=None,
    adc_res=None,
    adc_zero=None,
    init_value=None,
    checksum=None,
    block_size=None,
    record_only=False,
    header=True,
    delimiter=",",
    verbose=False,
):
    global df_CSV
    """
    Read a WFDB header file and return either a `Record` object with the
    record descriptors as attributes or write a record and header file.
    Parameters
    ----------
    file_name : str
        The name of the WFDB record to be read, without any file
        extensions. If the argument contains any path delimiter
        characters, the argument will be interpreted as PATH/BASE_RECORD.
        Both relative and absolute paths are accepted. If the `pn_dir`
        parameter is set, this parameter should contain just the base
        record name, and the files fill be searched for remotely.
        Otherwise, the data files will be searched for in the local path.
    fs : float
        This number can be expressed in any format legal for a Python input of
        floating point numbers (thus '360', '360.', '360.0', and '3.6e2' are
        all legal and equivalent). The sampling frequency must be greater than 0;
        if it is missing, a value of 250 is assumed.
    units : list, str
        This will be applied as the passed list unless a single str is passed
        instead - in which case the str will be assigned for all channels.
        This field can be present only if the ADC gain is also present. It
        follows the baseline field if that field is present, or the gain field
        if the baseline field is absent. The units field is a list of character
        strings that specifies the type of physical unit. If the units field is
        absent, the physical unit may be assumed to be 1 mV.
    fmt : list, str, optional
        This will be applied as the passed list unless a single str is passed
        instead - in which case the str will be assigned for all
        channels. A list of strings giving the WFDB format of each file used to
        store each channel. Accepted formats are: '80','212','16','24', and
        '32'. There are other WFDB formats as specified by:
        https://www.physionet.org/physiotools/wag/signal-5.htm
        but this library will not write (though it will read) those file types.
        Each field is an integer that specifies the storage format of the signal.
        All signals in a given group are stored in the same format. The most
        common format is format `16` (sixteen-bit amplitudes). The parameters
        `samps_per_frame`, `skew`, and `byte_offset` are optional fields, and
        if present, are bound to the format field. In other words, they may be
        considered as format modifiers, since they further describe the encoding
        of samples within the signal file.
    adc_gain : list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels.
        This field is a list of numbers that specifies the difference in
        sample values that would be observed if a step of one physical unit
        occurred in the original analog signal. For ECGs, the gain is usually
        roughly equal to the R-wave amplitude in a lead that is roughly parallel
        to the mean cardiac electrical axis. If the gain is zero or missing, this
        indicates that the signal amplitude is uncalibrated; in such cases, a
        value of 200 ADC units per physical unit may be assumed.
    baseline : list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels. This
        field can be present only if the ADC gain is also present. It is not
        separated by whitespace from the ADC gain field; rather, it is
        surrounded by parentheses, which delimit it. The baseline is an integer
        that specifies the sample value corresponding to 0 physical units. If
        absent, the baseline is taken to be equal to the ADC zero. Note that
        the baseline need not be a value within the ADC range; for example,
        if the ADC input range corresponds to 200-300 degrees Kelvin, the
        baseline is the (extended precision) value that would map to 0 degrees
        Kelvin.
    samps_per_frame : list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels.
        Normally, all signals in a given record are sampled at the (base)
        sampling frequency as specified by `fs`; in this case, the number of
        samples per frame is 1 for all signals, and this field is conventionally
        omitted. If the signal was sampled at some integer multiple, n, of the
        base sampling frequency, however, each frame contains n samples of the
        signal, and the value specified in this field is also n. (Note that
        non-integer multiples of the base sampling frequency are not supported).
    counter_freq : float, optional
        This field (a floating-point number, in the same format as `fs`) can be
        present only if `fs` is also present. Typically, the counter frequency
        may be derived from an analog tape counter, or from page numbers in a
        chart recording. If the counter frequency is absent or not positive,
        it is assumed to be equal to `fs`.
    base_counter : float, optional
        This field can be present only if the counter frequency is also present.
        The base counter value is a floating-point number that specifies the counter
        value corresponding to sample 0. If absent, the base counter value is
        taken to be 0.
    base_time : datetime.time, optional
        This field can be present only if the number of samples is also present.
        It gives the time of day that corresponds to the beginning of the
        record.
    base_date : datetime.date, optional
        This field can be present only if the base time is also present. It contains
        the date that corresponds to the beginning of the record.
    comments : list, optional
        A list of string comments to be written to the header file. Each string
        entry represents a new line to be appended to the bottom of the header
        file ('.hea').
    sig_name : list, optional
        A list of strings giving the signal name of each signal channel. This
        will be used for plotting the signal both in this package and
        LightWave. Note, this value will be used in preference to the CSV
        header, if applicable, to define custom signal names.
    dat_file_name : str, optional
        The name of the file in which samples of the signal are kept. Although the
        record name is usually part of the signal file name, this convention is
        not a requirement. Note that several signals can share the same file
        (i.e., they can belong to the same signal group); all entries for signals
        that share a given file must be consecutive, however. Note, the default
        behavior is to save the files in the current working directory, not the
        directory of the file being read.
    skew : list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels.
        Ideally, within a given record, samples of different signals with the
        same sample number are simultaneous (within one sampling interval).
        If this is not the case (as, for example, when a multitrack analog
        tape recording is digitized and the azimuth of the playback head does
        not match that of the recording head), the skew between signals can
        sometimes be determined (for example, by locating recorded waveform
        features with known time relationships, such as calibration signals).
        If this has been done, the skew field may be inserted into the header
        file to indicate the (positive) number of samples of the signal that
        are considered to precede sample 0. These samples, if any, are included
        in the checksum. (Note the checksum need not be changed if the skew field
        is inserted or modified).
    byte_offset : list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels.
        Normally, signal files include only sample data. If a signal file
        includes a preamble, however, this field specifies the offset in bytes
        from the beginning of the signal file to sample 0 (i.e., the length
        of the preamble). Data within the preamble is not included in the signal
        checksum. Note that the byte offset must be the same for all signals
        within a given group (use the skew field to correct for intersignal
        skew). This feature is provided only to simplify the task of reading
        signal files not generated using the WFDB library; the WFDB library
        does not support any means of writing such files, and byte offsets must
        be inserted into header files manually.
    adc_res: list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels.
        This field can be present only if the ADC gain is also present. It
        specifies the resolution of the analog-to-digital converter used to
        digitize the signal. Typical ADCs have resolutions between 8 and 16
        bits. If this field is missing or zero, the default value is 12 bits
        for amplitude-format signals, or 10 bits for difference-format signals
        (unless a lower value is specified by the format field).
    adc_zero: list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels.
        This field can be present only if the ADC resolution is also present.
        It is an integer that represents the amplitude (sample value) that
        would be observed if the analog signal present at the ADC inputs had
        a level that fell exactly in the middle of the input range of the ADC.
        For a bipolar ADC, this value is usually zero, but a unipolar (offset
        binary) ADC usually produces a non-zero value in the middle of its
        range. Together with the ADC resolution, the contents of this field
        can be used to determine the range of possible sample values. If this
        field is missing, a value of 0 is assumed.
    init_value : list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels.
        This field can be present only if the ADC zero is also present. It
        specifies the value of sample 0 in the signal, but is used only if the
        signal is stored in difference format. If this field is missing, a
        value equal to the ADC zero is assumed.
    checksum : list, optional
        This field can be present only if the initial value is also present. It
        is a 16-bit signed checksum of all samples in the signal. (Thus the
        checksum is independent of the storage format.) If the entire record
        is read without skipping samples, and the headers record line specifies
        the correct number of samples per signal, this field is compared against
        a computed checksum to verify that the signal file has not been corrupted.
        A value of zero may be used as a field placeholder if the number of
        samples is unspecified.
    block_size : list, int, optional
        This will be applied as the passed list unless a single int is passed
        instead - in which case the int will be assigned for all channels.
        This field can be present only if the checksum is present. This field
        is an integer and is usually 0. If the signal is stored in a file
        that must be read in blocks of a specific size, however, this field
        specifies the block size in bytes. (On UNIX systems, this is the case
        only for character special files, corresponding to certain tape and
        raw disk files. If necessary, the block size may be given as a negative
        number to indicate that the associated file lacks I/O driver support for
        some operations.) All signals belonging to the same signal group have
        the same block size.
    record_only : bool, optional
        Whether to only return the record information (True) or not (False).
        If false, this function will generate both a .dat and .hea file.
    header : bool, optional
        Whether to assume the CSV has a first line header (True) or not (False)
        which defines the signal names. If false, this function will generate
        either the signal names provided by `sig_name` or set `[ch_1, ch_2, ...]`
        as the default.
    delimiter : str, optional
        What to use as the delimiter for the file to separate data. The default
        if a comma (','). Other common delimiters are tabs ('\t'), spaces (' '),
        pipes ('|'), and colons (':').
    verbose : bool, optional
        Whether to print all the information read about the file (True) or
        not (False).
    Returns
    -------
    record : Record or MultiRecord, optional
        The WFDB Record or MultiRecord object representing the contents
        of the CSV file read.
    Notes
    -----
    CSVs should be in the following format:
    sig_1_name,sig_2_name,...
    sig_1_val_1,sig_2_val_1,...
    sig_1_val_2,sig_2_val_2,...
    ...,...,...
    Or this format if `header=False` is defined:
    sig_1_val_1,sig_2_val_1,...
    sig_1_val_2,sig_2_val_2,...
    ...,...,...
    The signal will be saved defaultly as a `p_signal` so both floats and
    ints are acceptable.
    Examples
    --------
    Create the header ('.hea') and record ('.dat') files, specifies both
    units to be 'mV'
    >>> csv_to_wfdb('sample-data/100.csv', fs=360, units='mV')
    Create the header ('.hea') and record ('.dat') files, change units for
    each signal
    >>> csv_to_wfdb('sample-data/100.csv', fs=360, units=['mV','kV'])
    Return just the record, note the use of lists to specify which values should
    be applied to each signal
    >>> csv_record = csv_to_wfdb('sample-data/100.csv', fs=360, units=['mV','mV'],
                                 fmt=['80',212'], adc_gain=[100,200],
                                 baseline=[1024,512], record_only=True)
    Return just the record, note the use of single strings and ints to specify
    when fields can be applied to all signals
    >>> csv_record = csv_to_wfdb('sample-data/100.csv', fs=360, units='mV',
                                 fmt=['80','212'], adc_gain=200, baseline=1024,
                                 record_only=True)
    """
    # NOTE: No need to write input checks here since the Record class should
    # handle them (except verifying the CSV input format which is for Pandas)
#    if header:
#        df_CSV = pd.read_csv(file_name, delimiter=delimiter)
#    else:
#        df_CSV = pd.read_csv(file_name, delimiter=delimiter, header=None)
    df_CSV=file_name
    if verbose:
        print("Successfully read CSV")
    # Extract the entire signal from the dataframe
    p_signal = df_CSV.values
    # The dataframe should be in (`sig_len`, `n_sig`) dimensions
    sig_len = p_signal.shape[0]
    if verbose:
        print("Signal length: {}".format(sig_len))
    n_sig = p_signal.shape[1]
    if verbose:
        print("Number of signals: {}".format(n_sig))
    # Check if signal names are valid and set defaults
    if not sig_name:
        if header:
            sig_name = df_CSV.columns.to_list()
            if any(map(str.isdigit, sig_name)):
                print(
                    "WARNING: One or more of your signal names are numbers, this "
                    "is not recommended:\n- Does your CSV have a header line "
                    "which defines the signal names?\n- If not, please set the "
                    "parameter 'header' to False.\nSignal names: {}".format(
                        sig_name
                    )
                )
        else:
            sig_name = ["ch_" + str(i) for i in range(n_sig)]
            if verbose:
                print("Signal names: {}".format(sig_name))

    # Set the output header file name to be the same, remove path
#    if os.sep in file_name:
#        file_name = file_name.split(os.sep)[-1]
    record_name = dat_file_name #file_name.replace(".csv", "")
    if verbose:
        print("Output header: {}.hea".format(record_name))

    # Replace the CSV file tag with DAT
#    dat_file_name = file_name.replace(".csv", ".dat")
    dat_file_name = [dat_file_name] * n_sig
    if verbose:
        print("Output record: {}".format(dat_file_name[0]))

    # Convert `units` from string to list if necessary
    units = [units] * n_sig if type(units) is str else units

    # Set the default `fmt` if none exists
    if not fmt:
        fmt = ["16"] * n_sig
    fmt = [fmt] * n_sig if type(fmt) is str else fmt
    if verbose:
        print("Signal format: {}".format(fmt))

    # Set the default `adc_gain` if none exists
    if not adc_gain:
        adc_gain = [200] * n_sig
    adc_gain = [adc_gain] * n_sig if type(adc_gain) is int else adc_gain
    if verbose:
        print("Signal ADC gain: {}".format(adc_gain))

    # Set the default `baseline` if none exists
    if not baseline:
        if adc_zero:
            baseline = [adc_zero] * n_sig
        else:
            baseline = [0] * n_sig
    baseline = [baseline] * n_sig if type(baseline) is int else baseline
    if verbose:
        print("Signal baseline: {}".format(baseline))

    # Convert `samps_per_frame` from int to list if necessary
    samps_per_frame = (
        [samps_per_frame] * n_sig
        if type(samps_per_frame) is int
        else samps_per_frame
    )

    # Convert `skew` from int to list if necessary
    skew = [skew] * n_sig if type(skew) is int else skew

    # Convert `byte_offset` from int to list if necessary
    byte_offset = (
        [byte_offset] * n_sig if type(byte_offset) is int else byte_offset
    )

    # Set the default `adc_res` if none exists
    if not adc_res:
        adc_res = [12] * n_sig
    adc_res = [adc_res] * n_sig if type(adc_res) is int else adc_res
    if verbose:
        print("Signal ADC resolution: {}".format(adc_res))

    # Set the default `adc_zero` if none exists
    if not adc_zero:
        adc_zero = [0] * n_sig
    adc_zero = [adc_zero] * n_sig if type(adc_zero) is int else adc_zero
    if verbose:
        print("Signal ADC zero: {}".format(adc_zero))

    # Set the default `init_value`
    # NOTE: Initial value (and subsequently the digital signal) won't be correct
    # unless the correct `baseline` and `adc_gain` are provided... this is just
    # the best approximation
    if not init_value:
        init_value = p_signal[0, :]
        init_value = baseline + (np.array(adc_gain) * init_value)
        init_value = [int(i) for i in init_value.tolist()]
    if verbose:
        print("Signal initial value: {}".format(init_value))

    # Set the default `checksum`
    if not checksum:
        checksum = [(np.sum(v) % 65536) for v in np.transpose(p_signal)]
    if verbose:
        print("Signal checksum: {}".format(checksum))

    # Set the default `block_size`
    if not block_size:
        block_size = [0] * n_sig
    block_size = [block_size] * n_sig if type(block_size) is int else block_size
    if verbose:
        print("Signal block size: {}".format(block_size))

    # Convert array to floating point
    p_signal = p_signal.astype("float64")

    # Either return the record or generate the record and header files
    # if requested
    if record_only:
        # Create the record from the input and generated values
        record = Record(
            record_name=record_name,
            n_sig=n_sig,
            fs=fs,
            samps_per_frame=samps_per_frame,
            counter_freq=counter_freq,
            base_counter=base_counter,
            sig_len=sig_len,
            base_time=base_time,
            base_date=base_date,
            comments=comments,
            sig_name=sig_name,
            p_signal=p_signal,
            d_signal=None,
            e_p_signal=None,
            e_d_signal=None,
            file_name=dat_file_name,
            fmt=fmt,
            skew=skew,
            byte_offset=byte_offset,
            adc_gain=adc_gain,
            baseline=baseline,
            units=units,
            adc_res=adc_res,
            adc_zero=adc_zero,
            init_value=init_value,
            checksum=checksum,
            block_size=block_size,
        )
        if verbose:
            print("Record generated successfully")
        return record

    else:
        # Write the information to a record and header file
        wrsamp(
            record_name=record_name,
            fs=fs,
            units=units,
            sig_name=sig_name,
            p_signal=p_signal,
            fmt=fmt,
            adc_gain=adc_gain,
            baseline=baseline,
            comments=comments,
            base_time=base_time,
            base_date=base_date,
        )
        if verbose:
            print("File generated successfully")
            
cwd='/fs/ess/scratch/PAS2164/TempData2/TempWF/'
os.chdir(cwd)
masterFile="/fs/ess/scratch/PAS2164/TempData2/masterFile.txt"
masterFinish="/fs/ess/scratch/PAS2164/TempData2/masterFinish.txt"

pathList=[]
writeMaster=open(masterFile)
for l in writeMaster:
    pathList.append(l)
writeMaster.close()

for folder in os.listdir('/fs/ess/scratch/PAS2164/TempData2/Temp4/'):
    if '_SUCCESS' in os.listdir(os.path.join('/fs/ess/scratch/PAS2164/TempData2/Temp4/',folder)):
        if str(folder+'\n') not in pathList
            writeMaster=open(masterFile,'a')
            writeMaster.write((folder+'\n'))
            writeMaster.close()
            errorCode=0
            dfWave=pd.read_parquet(os.path.join('/fs/ess/scratch/PAS2164/TempData2/Temp4/',folder), engine='pyarrow')
                #errorCode="Failed To Load"
            if errorCode==0:
                try:
                    dfWaveDict={}
                    dfWaveDict['csID']=dfWave.csID.mode()[0]
                    dfWave['DateTime']=pd.to_datetime(dfWave['offsetDate'].astype(str)+" "+dfWave['offsetTime'])
                except:
                    errorCode="Empty Dataset"
            if errorCode==0:
                dfTime=pd.DataFrame(dfWave['DateTime'].unique(), columns=['DateTime']).sort_values(by=['DateTime'])
                dfTime=dfTime.reset_index()
                dfTime['dt2']=dfTime['DateTime'].shift(1)
                dfTime['ddt']=(dfTime.DateTime-dfTime.dt2)
                n=0
                newTime=[]
                for index, row in dfTime.iterrows():
                    if row['ddt'].seconds > 3:
                        n=0
                    if n==0:
                        lastAnchorDateTime=row['DateTime']
                    newTime.append([lastAnchorDateTime,index])
                    n=n+1
                dfTime=dfTime.merge(pd.DataFrame(newTime, columns=['AnchorTime','index3']), left_index = True, right_on = 'index3')
                dfTime=dfTime[['DateTime','AnchorTime']]

                dfWave=dfTime.merge(dfWave,left_on='DateTime',right_on='DateTime',how='outer')


                dfWaveDict={}

                #time
                for mgTime in dfTime['AnchorTime'].unique():
                    dfA=dfWave[dfWave['AnchorTime']==mgTime]
                    dfWaveDict[mgTime]={}
                    dfWaveDict[mgTime]['csID']=dfA.csID.mode()[0]
                    dfWaveDict[mgTime]['csBedID']=dfA.csBedID.mode()[0]
                    for mgName in dfA['mgname'].unique():
                        dfT=dfA[dfA['mgname']==mgName]
                        dfWaveDict[mgTime][mgName]={}
                        dfWaveDict[mgTime][mgName]['mgGain']=(dfT.mgGain.mode()[0])
                        dfWaveDict[mgTime][mgName]['mguom']=(dfT.mguom.mode()[0])
                        dfWaveDict[mgTime][mgName]['mgsite']=(dfT.mgsite.mode()[0])
                        dfWaveDict[mgTime][mgName]['mgscale']=(dfT.mgscale.mode()[0])
                        dfWaveDict[mgTime][mgName]['mginvalid']=(dfT.mginvalid.mode()[0])
                        dfWaveDict[mgTime][mgName]['mgmissing']=(dfT.mgmissing.mode()[0])
                        dfWaveDict[mgTime][mgName]['mgPoints']=(dfT.mgPoints.mode()[0])
                        dfWaveDict[mgTime][mgName]['mgPointsBytes']=(dfT.mgPointsBytes.mode()[0])
                        dfWaveDict[mgTime][mgName]['mgMin']=(dfT.mgMin.mode()[0])
                        dfWaveDict[mgTime][mgName]['mgMax']=(dfT.mgMax.mode()[0])
                        dfWaveDict[mgTime][mgName]['mgOffset']=(dfT.mgOffset.mode()[0])
                        dfT=dfT[['mgPoints','DateTime','mgwave']]
                        baseHz=(dfT.mgPoints.mode()[0])
                        dfT['mgPointsNext']=dfT['mgPoints'].shift(-1)
                        dfT['mgPointsPrev']=dfT['mgPoints'].shift(1)
                        dfT['mgPointsNext2']=dfT['mgPoints'].shift(-2)
                        dfT['mgPointsPrev2']=dfT['mgPoints'].shift(2)
                        orderMod=[]
                        for index, row in dfT.iterrows():
                            if row['mgPoints'] == str(baseHz):
                                orderMod.append([index,0])
                            if row['mgPoints'] != str(baseHz):
                                try:
                                    if ((int(row['mgPoints'])+int(row['mgPointsNext']))%int(baseHz)==0 or (int(row['mgPoints'])+int(row['mgPointsNext2']))%int(baseHz)==0):
                                        orderMod.append([index,1])
                                    elif ((int(row['mgPoints'])+int(row['mgPointsPrev']))%int(baseHz)==0 or (int(row['mgPoints'])+int(row['mgPointsPrev2']))%int(baseHz)==0):
                                        orderMod.append([index,-1])
                                    else: orderMod.append([index,0])
                                except:
                                    orderMod.append([index,0])
                        dfT=dfT.merge(pd.DataFrame(orderMod, columns=['index2','Mod']), left_index = True, right_on = 'index2')
                        dfT=dfT.sort_values(by=["DateTime",'Mod'])
                        dfT=dfT[["DateTime","Mod","mgwave"]]
                        invalidString='-32752'
                        for x in range(int(baseHz)-1):
                            invalidString=invalidString+'^-32752'
                        dfT=dfT.fillna(value={"mgPoints": baseHz, "mgwave": invalidString})
                        dfT['mgwave']=dfT['mgwave'].str.split('^').apply(lambda x: [int(i) for i in x]) 
                        dfT=(dfT.mgwave.apply(pd.Series))\
                                    .merge(dfT, right_index = True, left_index = True)\
                                    .drop(["mgwave"], axis = 1)\
                                    .melt(id_vars = ['DateTime','Mod'], value_name = "mgwave")
                        dfT=dfT.dropna()
                        dfT=dfT.sort_values(by=['DateTime','Mod','variable'])
                        dfT=dfT.reset_index()
                        dfWaveDict[mgTime][mgName]['waveform']=dfT.mgwave.astype(int)
                        dfWaveDict[mgTime][mgName]['waveform']=dfWaveDict[mgTime][mgName]['waveform'].rename(mgName)

                for anchor in dfWaveDict:
                    dat_file_name=str(dfWaveDict[anchor]["csID"])+"^"+str(dfWaveDict[anchor]["csBedID"])+"^"+str(anchor)
                    dat_file_name=dat_file_name.split('.')[0].replace(":","")
                    base_time=pd.to_datetime(anchor).time()
                    base_date=pd.to_datetime(anchor).date()
                    print(dat_file_name)
                    units=[]
                    file_name=[]
                    adc_gain=[]
                    baseline=[]
                    sampleRateList=[]
                    sig_name=[]
                    adc_zero_lists=[]
                    byte_offset=[]
                    fmt=[]
                    for device in dfWaveDict[anchor]:
                        if device not in ["csID","csBedID"]:
                            sig_name.append(device)
                            units.append(dfWaveDict[anchor][device]['mguom'])
                            file_name.append(dfWaveDict[anchor][device]['waveform'])
                            adc_gain.append(int(float(dfWaveDict[anchor][device]['mgGain'])))
                            baseline.append(int(float(dfWaveDict[anchor][device]['mgOffset'])))
                            sampleRateList.append(int(dfWaveDict[anchor][device]['mgPoints']))
                            byte_offset.append(dfWaveDict[anchor][device]['mgPointsBytes'])
                            adc_zero_lists.append(dfWaveDict[anchor][device]['mginvalid'])
                            adc_zero_lists.append(dfWaveDict[anchor][device]['mgmissing'])
                            fmt.append('32')
                    fs=math.gcd(*sampleRateList)

                    samps_per_frame=[]
                    for fsl in sampleRateList:
                        samps_per_frame.append(fsl/fs)
                    invlist=[]
                    for miv in adc_zero_lists:
                        for val in (miv.split(",")):
                            invlist.append(val)
                    adc_zero=list(set(invlist))
                    test=pd.DataFrame(file_name)
                    test=pd.DataFrame.transpose(test)
                    pandas_to_wfdb(test,fs,units,fmt=fmt,dat_file_name=dat_file_name,adc_gain=adc_gain,header=False, delimiter=',',base_time=base_time,base_date=base_date,sig_name=sig_name,adc_zero=adc_zero,samps_per_frame=samps_per_frame,baseline=baseline)
                print(folder)
            if errorCode!=0:
                print(errorCode)
            masterFin=open(masterFinish,'a')
            masterFin.write((folderPath+'\n'))
            masterFin.close()